In [8]:
import streamlit as st
from PIL import Image
from openai import OpenAI
import io
import pytesseract
import pandas as pd
import awswrangler as wr
import boto3
import base64
import json
import os
from urllib.parse import urlparse
import streamlit.components.v1 as components
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network
from taxon_parser import TaxonParser, UnparsableNameException
import math
import numpy as np
import socket
from botocore.exceptions import EndpointConnectionError, ConnectTimeoutError
import json
from pprint import pprint
from pathlib import Path

In [15]:
DATA_DIR = Path('./data')

DOWNLOADS_DIR = Path('/Users/ben/Downloads')

In [23]:


with open(DOWNLOADS_DIR / 'opends-ccc9262c927b48749ef261364ab86dfa.json') as f:
    data = json.load(f)
    # json_data.close()
    pprint(data)

{'@context': {'@vocab': 'https://schema.org/',
              'dwc': 'http://rs.tdwg.org/dwc/terms/',
              'gbif': 'https://www.gbif.org/species/',
              'geo': 'http://www.w3.org/2003/01/geo/wgs84_pos#',
              'ods': 'https://github.com/DiSSCo/openDS',
              'wd': 'https://www.wikidata.org/wiki/'},
 '@type': 'ods:DigitalSpecimen',
 'ods:basisOfRecord': 'PreservedSpecimen',
 'ods:created': '2024-08-09T02:06:18Z',
 'ods:license': 'CC-BY 4.0',
 'ods:locality': {'@type': 'Place', 'name': 'United States'},
 'ods:modified': '2024-08-09T02:06:18Z',
 'ods:physicalSpecimenCollection': {'@id': 'https://scientific-collections.gbif.org/institution/219261fd-d235-4d52-acd6-ddc79dc7e870',
                                    '@type': 'Organization',
                                    'dwc:country': 'United States',
                                    'dwc:countryCode': 'US',
                                    'dwc:institutionCode': 'Harvard University',
             

In [36]:
g = Network(height='400px', width='80%', heading='')
g.add_node(0, label='Specimen', color='#f7b5ca')

# Plot the current specimen node
if locality := data.get('ods:locality'):
    g.add_node(1, label=locality['name'], color='#f5c669', title="Country")
    # Add edges
    g.add_edge(0, 1, color='black', label='locality')

# Plot the current specimen node
if data.get('ods:physicalSpecimenCollection'):
    g.add_node(2, label=data['ods:physicalSpecimenCollection']['dwc:institutionName'], color='#82b6fa', title="Institution")
    g.add_edge(0, 2, color='black', label='collection')
    
    # Add edges
    if institution_country := data['ods:physicalSpecimenCollection'].get('dwc:country'):

        institution_country = 'France'
        
        if data.get('ods:locality').get('name') == institution_country:
            g.add_edge(2, 1, color='black')
        else:
            g.add_node(3, label=institution_country, color='#f5c669', title="Country")
            g.add_edge(2, 3, color='black')
            
        print(institution_country)

    # g.add_edge(0, 1, color='black', label='locality')
    g.add_edge(0, 1, color='black', label='locality')
        
if data.get('ods:scientificName'):
    g.add_node(4, label=data['ods:scientificName']['dwc:scientificName'], color='#befa82', title=data['ods:scientificName']['dwc:taxonRank'].title())
    g.add_edge(0, 4, color='black', label='determination')

# Generate and show the network
html_file = 'example.html'
g.save_graph(html_file)

from IPython.display import display, HTML
  
# Read the contents of the HTML file
with open(html_file, 'r') as file:
    html_content = file.read()
    
# Display the HTML content
display(HTML(html_content))

France
